In [1]:
# sample and concatinate 10s snipits from 300 games and save using ffmpeg
import os


fp = '/Volumes/BBall_Data_23X_pt2/HUDL_basketball_data_pt2/22-23/misc-replays'
video_fps = []

for root, dirs, files in os.walk(fp):
    for file in files:
        if file.endswith('mp4') or file.endswith('avi'):
            full_path = os.path.join(root, file)
            video_fps.append(full_path)

In [5]:
import os
import random
import subprocess

def extract_random_clip(video_path, num_frames, output_path):
    # Get the frame rate of the video
    probe_cmd = ['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 
                 'stream=r_frame_rate', '-of', 'default=noprint_wrappers=1:nokey=1', video_path]
    result = subprocess.run(probe_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    if result.returncode != 0:
        raise Exception(f"ffprobe error: {result.stderr.strip()}")

    # Calculate the duration in seconds for the given number of frames
    frame_rate = eval(result.stdout.strip())  # Frame rate as a fraction, e.g., '30/1'
    duration_seconds = num_frames / frame_rate  # Duration for num_frames frames
    
    # Get the total duration of the video in seconds
    probe_duration_cmd = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration',
                          '-of', 'default=noprint_wrappers=1:nokey=1', video_path]
    total_duration_result = subprocess.run(probe_duration_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    total_duration = float(total_duration_result.stdout.strip())

    assert total_duration > duration_seconds, "Video is too short for clipping."

    # Generate a random start time
    max_start_time = total_duration - duration_seconds
    start_time = random.uniform(0, max_start_time)

    # Build the ffmpeg command for extracting the clip
    ffmpeg_cmd = [
        'ffmpeg', '-y', '-ss', str(start_time), '-t', str(duration_seconds), 
        '-i', video_path, '-c:v', 'libx264', '-c:a', 'aac', '-strict', 'experimental', 
        output_path
    ]
    subprocess.run(ffmpeg_cmd, check=True)

def concatenate_clips(clip_paths, output_path):
    """
    Concatenates multiple video clips into a single video file.

    Args:
    clip_paths (list of str): A list of paths to the video clips.
    output_path (str): The path where the concatenated video will be saved.

    Returns:
    None
    """
    # Create a temporary file to list all clip paths
    with open('filelist.txt', 'w') as file:
        for path in clip_paths:
            file.write(f"file '{path}'\n")

    # Build the ffmpeg command for concatenation
    ffmpeg_cmd = [
        'ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'filelist.txt', 
        '-c', 'copy', output_path
    ]
    subprocess.run(ffmpeg_cmd, check=True)
    os.remove('filelist.txt')  # Clean up temporary file

def process_videos(video_paths, num_clips, clip_duration, output_video):
    """
    Processes a list of video paths to create a single video composed of random clips.

    Args:
    video_paths (list of str): The list of video paths.
    num_clips (int): Number of random clips to extract.
    clip_duration (int): Duration of each clip in seconds.
    output_video (str): The output video path.

    Returns:
    None
    """
    assert len(video_paths) >= num_clips, "Not enough video paths provided."

    selected_paths = random.sample(video_paths, num_clips)
    clip_files = []

    for i, path in enumerate(selected_paths):
        try:
            output_path = f"temp_clip_{i}.mp4"
            extract_random_clip(path, clip_duration, output_path)
            clip_files.append(output_path)
        except:
            print(f"Error: could not process clip @ {path}")

    concatenate_clips(clip_files, output_video)

    # Cleanup temporary clips
    for clip in clip_files:
        os.remove(clip)

process_videos(video_fps, 300, 10, 'multi-league-30x10.mp4')

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57